In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [11]:
df = pd.read_csv("classification_trian_dataset.csv")
df.head()

,Unnamed: 0,Title,Product,Category,Geography,Frequency,Reference Period,Release Date,Table No,Download URL,Data Source,Description,id,search_text
0,28,Statement 1: Average Monthly Per Capita Consum...,HCES,National Sample Surveu,All India,As and when requirement arises,2023-24,31-12-2024,HCESAFY24001ANN,https://esankhyiki.mospi.gov.in/datacatalogue/...,National Sample Survey Office,The Household Consumption Expenditure Survey (...,28,statement 1 average monthly per capita consump...
1,28,command 1 : Average Monthly Per Capita Consump...,HCES,National Sample Surveu,All India,As and when requirement arises,2023-24,31-12-2024,HCESAFY24001ANN,https://esankhyiki.mospi.gov.in/datacatalogue/...,National Sample Survey Office,The Household Consumption Expenditure Survey (...,28,assertion 1 intermediate monthly per capita pu...
2,28,assertion 1 : Average Monthly Per Capita Consu...,HCES,National Sample Surveu,All India,As and when requirement arises,2023-24,31-12-2024,HCESAFY24001ANN,https://esankhyiki.mospi.gov.in/datacatalogue/...,National Sample Survey Office,The Household Consumption Expenditure Survey (...,28,command 1 ordinary monthly per capita uptake s...
3,28,affirmation 1 : Average Monthly Per Capita Con...,HCES,National Sample Surveu,All India,As and when requirement arises,2023-24,31-12-2024,HCESAFY24001ANN,https://esankhyiki.mospi.gov.in/datacatalogue/...,National Sample Survey Office,The Household Consumption Expenditure Survey (...,28,assertion 1 mean monthly per capita intake exp...
4,28,statement 1 : Average Monthly Per Capita Consu...,HCES,National Sample Surveu,All India,As and when requirement arises,2023-24,31-12-2024,HCESAFY24001ANN,https://esankhyiki.mospi.gov.in/datacatalogue/...,National Sample Survey Office,The Household Consumption Expenditure Survey (...,28,affirmation 1 mediocre monthly per capita usan...


# Try Machine Learning Techniques

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB  
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import joblib
from sentence_transformers import SentenceTransformer

In [14]:
# ---- Step 1: Load Model & Prepare Data ----
model_name = 'all-MiniLM-L6-v2'
embedder = SentenceTransformer(model_name)

X = df['search_text'].tolist()
y = df['Product']

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)
num_classes, le.classes_

(10,
 array(['ASI', 'ASUSE', 'CAMS', 'CPI', 'HCES', 'IIP', 'MIS', 'NAS', 'PLFS',
        'WMI'], dtype=object))

In [ ]:
for idx, label in enumerate(le.classes_):
    print(f"{idx} → {label}")

0 → ASI
1 → ASUSE
2 → CAMS
3 → CPI
4 → HCES
5 → IIP
6 → MIS
7 → NAS
8 → PLFS
9 → WMI


In [ ]:
model_name = 'all-MiniLM-L6-v2'
embedder = SentenceTransformer(model_name)

X = df['search_text'].tolist()
y = df['Product']

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)

# Create dense embeddings (384-d vectors)
X_embeddings = embedder.encode(X, show_progress_bar=True)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y_encoded, test_size=0.2, random_state=42)

models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "LinearSVC": LinearSVC(),
    "RandomForest": RandomForestClassifier(n_estimators=50, n_jobs=-1),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

# Train & Evaluate
results = []

for name, model in models.items():
    print(f"\n{name} Training...")

    model.fit(X_train, y_train)

    # Save model & calculate size
    file_path = f"models/{name}_embedding_model.joblib"
    joblib.dump(model, file_path)
    model_size_kb = os.path.getsize(file_path) / 1024  # in KB

    # Inference Time (avg over batches of 512)
    batch_size = 512
    total_time = 0
    num_batches = int(np.ceil(len(X_test) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(X_test))
        X_batch = X_test[start_idx:end_idx]

        start = time.time()
        _ = model.predict(X_batch)
        total_time += time.time() - start

    avg_inference_time = (total_time * 1000 / num_batches)  # ms

    preds = model.predict(X_test)

    # Accuracy & Report
    acc = accuracy_score(y_test, preds)
    print(f"Accuracy: {acc:.4f}")
    print(f"Model size: {model_size_kb:.2f} KB")
    print(f"Average inference time per batch: {avg_inference_time:.4f} ms")
    print(classification_report(le.inverse_transform(y_test), le.inverse_transform(preds)))

    # Store results
    results.append({
        "Model": name,
        "Accuracy": acc,
        "Model Size (KB)": model_size_kb,
        "Avg Inference Time (ms)": avg_inference_time
    })

summary_df = pd.DataFrame(results)
print("\n--- Summary ---")
print(summary_df)


Batches: 100%|██████████| 223/223 [00:11<00:00, 19.51it/s]



LogisticRegression Training...
Accuracy: 1.0000
Model size: 30.98 KB
Average inference time per batch: 1.6755 ms
              precision    recall  f1-score   support

         ASI       1.00      1.00      1.00        67
       ASUSE       1.00      1.00      1.00       115
        CAMS       1.00      1.00      1.00       129
         CPI       1.00      1.00      1.00        47
        HCES       1.00      1.00      1.00       232
         IIP       1.00      1.00      1.00       224
         MIS       1.00      1.00      1.00       124
         NAS       1.00      1.00      1.00       286
        PLFS       1.00      1.00      1.00       101
         WMI       1.00      1.00      1.00        99

    accuracy                           1.00      1424
   macro avg       1.00      1.00      1.00      1424
weighted avg       1.00      1.00      1.00      1424


LinearSVC Training...
Accuracy: 1.0000
Model size: 30.85 KB
Average inference time per batch: 1.3329 ms
              precisio

d:\OneDrive - iitgn.ac.in\Desktop\Projects\SmartSearchify-AI-Enabled-Semantic-Search-for-eSankhyiki-Portal\env\lib\site-packages\xgboost\training.py:183: UserWarning: [21:55:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.9993
Model size: 880.94 KB
Average inference time per batch: 2.1706 ms
              precision    recall  f1-score   support

         ASI       1.00      1.00      1.00        67
       ASUSE       1.00      0.99      1.00       115
        CAMS       1.00      1.00      1.00       129
         CPI       1.00      1.00      1.00        47
        HCES       1.00      1.00      1.00       232
         IIP       1.00      1.00      1.00       224
         MIS       0.99      1.00      1.00       124
         NAS       1.00      1.00      1.00       286
        PLFS       1.00      1.00      1.00       101
         WMI       1.00      1.00      1.00        99

    accuracy                           1.00      1424
   macro avg       1.00      1.00      1.00      1424
weighted avg       1.00      1.00      1.00      1424


--- Summary ---
                Model  Accuracy  Model Size (KB)  Avg Inference Time (ms)
0  LogisticRegression  1.000000        30.975586                 1.

# Neural Networks

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import pandas as pd
import numpy as np

In [ ]:
X = df['search_text'].tolist()
y = df['Product']

# Label encoding
label_enc = LabelEncoder()
y_encoded = label_enc.fit_transform(y)
num_classes = len(label_enc.classes_)

# Generate embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")
X_embed = embedder.encode(X, show_progress_bar=True)  # shape: (n_samples, 384)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_embed, y_encoded, test_size=0.2, random_state=42)

class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.dropout(self.relu(self.fc1(x)))
        return self.fc2(x)

input_dim = X_embed[0].shape[0]  
model = TextClassifier(input_dim=input_dim, hidden_dim=256, output_dim=num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/10], Loss: {total_loss / len(train_loader):.4f}")

# Evaluation 
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

print(f"Test Accuracy: {accuracy_score(all_labels, all_preds):.4f}")
print(classification_report(all_labels, all_preds, target_names=label_enc.classes_))

Batches: 100%|██████████| 223/223 [00:11<00:00, 19.23it/s]


Epoch [1/10], Loss: 1.2677
Epoch [2/10], Loss: 0.1384
Epoch [3/10], Loss: 0.0340
Epoch [4/10], Loss: 0.0169
Epoch [5/10], Loss: 0.0105
Epoch [6/10], Loss: 0.0072
Epoch [7/10], Loss: 0.0052
Epoch [8/10], Loss: 0.0039
Epoch [9/10], Loss: 0.0032
Epoch [10/10], Loss: 0.0026
Test Accuracy: 1.0000
              precision    recall  f1-score   support

         ASI       1.00      1.00      1.00        67
       ASUSE       1.00      1.00      1.00       115
        CAMS       1.00      1.00      1.00       129
         CPI       1.00      1.00      1.00        47
        HCES       1.00      1.00      1.00       232
         IIP       1.00      1.00      1.00       224
         MIS       1.00      1.00      1.00       124
         NAS       1.00      1.00      1.00       286
        PLFS       1.00      1.00      1.00       101
         WMI       1.00      1.00      1.00        99

    accuracy                           1.00      1424
   macro avg       1.00      1.00      1.00      1424
wei

 Try of Title Embedding

In [ ]:
X = df['Title'].tolist()
y = df['Product']

# Label encoding
label_enc = LabelEncoder()
y_encoded = label_enc.fit_transform(y)
num_classes = len(label_enc.classes_)

# Generate embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")
X_embed = embedder.encode(X, show_progress_bar=True)  # shape: (n_samples, 384)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_embed, y_encoded, test_size=0.2, random_state=42)

class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.dropout(self.relu(self.fc1(x)))
        return self.fc2(x)

input_dim = X_embed[0].shape[0]  
model = TextClassifier(input_dim=input_dim, hidden_dim=256, output_dim=num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/10], Loss: {total_loss / len(train_loader):.4f}")

# Evaluation 
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

print(f"Test Accuracy: {accuracy_score(all_labels, all_preds):.4f}")
print(classification_report(all_labels, all_preds, target_names=label_enc.classes_))

Batches: 100%|██████████| 223/223 [00:04<00:00, 55.40it/s]


Epoch [1/10], Loss: 1.4749
Epoch [2/10], Loss: 0.4624
Epoch [3/10], Loss: 0.2013
Epoch [4/10], Loss: 0.1214
Epoch [5/10], Loss: 0.0848
Epoch [6/10], Loss: 0.0627
Epoch [7/10], Loss: 0.0481
Epoch [8/10], Loss: 0.0381
Epoch [9/10], Loss: 0.0316
Epoch [10/10], Loss: 0.0273
Test Accuracy: 0.9923
              precision    recall  f1-score   support

         ASI       1.00      1.00      1.00        67
       ASUSE       1.00      1.00      1.00       115
        CAMS       0.98      0.99      0.99       129
         CPI       1.00      1.00      1.00        47
        HCES       1.00      1.00      1.00       232
         IIP       1.00      1.00      1.00       224
         MIS       0.95      1.00      0.97       124
         NAS       1.00      1.00      1.00       286
        PLFS       0.99      0.96      0.97       101
         WMI       1.00      0.95      0.97        99

    accuracy                           0.99      1424
   macro avg       0.99      0.99      0.99      1424
wei